In [ ]:
# Useful for debugging
%load_ext autoreload
%autoreload 2

 # xopt CNSGA2 function example
 
Experimental, using `pymoo` as a backend. 

In [ ]:
from xopt.cmoo import cnsga2
import json
import numpy as np
import matplotlib.pyplot as plt
from time import sleep

In [ ]:
NAME = 'TNK'
BOUND_LOW, BOUND_UP = [0.0, 0.0], [3.14159, 3.14159]  


X_RANGE = [0, 1.4]
Y_RANGE = [0, 1.4]

# Pure number version
def TNK(individual):    
    x1=individual[0]
    x2=individual[1]
    objectives =  (x1, x2)
    constraints = (x1**2+x2**2-1.0 - 0.1*np.cos(16*np.arctan2(x1, x2)), 0.5-(x1-0.5)**2-(x2-0.5)**2 )
    return objectives, constraints

# labeled version
def evaluate_TNK(inputs):
    
    info = {'some':'info', 'about':['the', 'run']}
    ind = [inputs['x1'], inputs['x2']]
    objectives, constraints = TNK(ind)    
    
    if 'sleeptime' in inputs:
        sleep(inputs['sleeptime'])
        x = np.random.random()
        if x < 0.3:
            sleep(inputs['sleeptime']*5)
    
    outputs = {'y1':objectives[0], 'y2':objectives[1], 'c1':constraints[0], 'c2':constraints[1]}
    
    return outputs

VOCS = {
    'name':'TNK_test',
    
    'variables': {
        'x1':[0, 3.14159],
        'x2':[0, 3.14159]
    },
    'objectives':{
        'y1':'MINIMIZE',
        'y2':'MINIMIZE'
        
    },
    'constraints':{
        'c1': ['GREATER_THAN', 0],
        'c2': ['GREATER_THAN', 0]
        
    },
    'constants':{'a':'dummy_constant'},
    'linked_variables':{'x9':'x1'}
    
    
}
TNK([0,0]), evaluate_TNK(dict(x1=1,x2=2, sleeptime=1))

In [ ]:
# Pick a parallel method

from concurrent.futures import ProcessPoolExecutor as PoolExecutor
#from concurrent.futures import ThreadPoolExecutor as PoolExecutor
executor = PoolExecutor()
#executor=None

In [ ]:
problem, res = cnsga2(executor, vocs=VOCS, evaluate_f=evaluate_TNK,
            max_generations = 20, population_size=64, output_path=None,
            rolling_evaluate=True,
            verbose=True) 

In [ ]:
pop_data = res.pop

In [ ]:
problem2, res2 = cnsga2(executor, vocs=VOCS, population=pop_data, evaluate_f=evaluate_TNK,
            max_generations = 40, population_size=64, output_path=None,
            rolling_evaluate=True,                                       
            verbose=True) 

In [ ]:
from pymoo.visualization.scatter import Scatter
plot = Scatter()
plot.add(problem.pareto_front(), plot_type="line", color="black", alpha=0.7)
plot.add(res2.F, color="red")
plot.show()

# Dask

In [ ]:
from distributed import Client
client = Client()
client 

In [ ]:
# Add sleep
VOCS['constants']['sleeptime'] = 1

In [ ]:
%%time
# Non-rolling
problem3, res3 = cnsga2(client, vocs=VOCS, population=None, evaluate_f=evaluate_TNK,
            max_generations = 10, population_size=8, output_path=None,
            rolling_evaluate=False,                                       
            verbose=True) 

In [ ]:
%%time
# Rolling
problem3, res3 = cnsga2(client, vocs=VOCS, population=None, evaluate_f=evaluate_TNK,
            max_generations = 10, population_size=8, output_path=None,
            rolling_evaluate=True,                                       
            verbose=True) 